In [6]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from card_detector import CardDetector, fit
from dataloader_utils import MTGCardsDataset, get_transform_pipe
from config import *

In [7]:
anchor_boxes = torch.Tensor([[198.27963804, 206.74086672],
       [129.59395666, 161.90171490],
       [161.65437828, 232.34624509]
]) # Anchor boxes acquired from k-means clustering of the dataset

model = CardDetector(img_dims= (CONFIG["dataset"]["img_w"], CONFIG["dataset"]["img_h"]),
                      anchor_boxes=anchor_boxes,
                        num_anchors_per_cell=CONFIG["dataset"]["num_anchors_per_cell"],
                          loss_fn=nn.L1Loss(reduction="sum")
                          )

dummy_image = torch.randn(1, 3, CONFIG["dataset"]["img_w"], CONFIG["dataset"]["img_h"]) # create a dummy image to pass to the model and get feature map size
feature_map_shape = model.feature_extractor(dummy_image).shape
feature_map_dims = (feature_map_shape[-2], feature_map_shape[-1])


transform_pipe = get_transform_pipe(img_w=CONFIG["dataset"]["img_w"], img_h=CONFIG["dataset"]["img_h"])

train_dataset = MTGCardsDataset(annotations_file=CONFIG["dataset"]["annotations_file"],
                                img_dir=CONFIG["dataset"]["img_dir"], #TODO: change directory when we have the actual data
                                  anchor_boxes=model.anchor_boxes,
                                    feature_map_dims=feature_map_dims,
                                      img_dims= (CONFIG["dataset"]["img_w"], CONFIG["dataset"]["img_h"]),
                                        num_anchors_per_cell=CONFIG["dataset"]["num_anchors_per_cell"],
                                          num_max_boxes=1,
                                            transform=transform_pipe
                                            )
train_dataloader = DataLoader(train_dataset, batch_size=CONFIG["dataloader"]["batch_size"])

val_dataset = MTGCardsDataset(annotations_file=CONFIG["dataset"]["annotations_file"],
                                img_dir=CONFIG["dataset"]["img_dir"], #TODO: change directory when we have the actual data
                                  anchor_boxes=model.anchor_boxes,
                                    feature_map_dims=feature_map_dims,
                                      img_dims= (CONFIG["dataset"]["img_w"], CONFIG["dataset"]["img_h"]),
                                        num_anchors_per_cell=CONFIG["dataset"]["num_anchors_per_cell"],
                                          num_max_boxes=1,
                                            transform=transform_pipe
                                            )
val_dataloader = DataLoader(val_dataset, batch_size=CONFIG["dataloader"]["batch_size"]) 

c:\Users\Leonard\anaconda3\envs\pytorch_nn_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Leonard\anaconda3\envs\pytorch_nn_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
opt = torch.optim.Adam(params=model.parameters(), lr=CONFIG["optimizer"]["lr"])
fit(model=model, num_epochs=50, optimizer=opt, train_dataloader= train_dataloader, val_dataloader= val_dataloader, device= ("cuda" if torch.cuda.is_available() else "cpu"))

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 0



  2%|▏         | 1/50 [00:04<03:31,  4.32s/it]

Looked at 0 Batches	---	0/2 Samples
Train loss: 1.9076826572418213 | Val Loss: 1.9076826572418213 | Val Accuracy: 0.0
Epoch: 1



  4%|▍         | 2/50 [00:04<01:34,  1.97s/it]

...

Looked at 0 Batches	---	0/2 Samples
Train loss: 0.09736550599336624 | Val Loss: 0.09736550599336624 | Val Accuracy: 0.0
Epoch: 48



 98%|█████████▊| 49/50 [00:27<00:00,  1.99it/s]

Looked at 0 Batches	---	0/2 Samples
Train loss: 0.09471507370471954 | Val Loss: 0.09471507370471954 | Val Accuracy: 0.0
Epoch: 49



100%|██████████| 50/50 [00:27<00:00,  1.80it/s]

Looked at 0 Batches	---	0/2 Samples
Train loss: 0.09219635277986526 | Val Loss: 0.09219635277986526 | Val Accuracy: 0.0
